In [ ]:
# To find ECD Control
import time
import sys 
sys.path.append(r'C:\Users\Chakram_Meas\Documents\GitHub\Multimode-Conditional-Displacements\hpc_runs\single_mode_ecd\class_description')
sys.path.append(r'C:\Users\Chakram_Meas\Documents\GitHub')

In [ ]:
import numpy as np
from qutip import *
from ECD_batch_optimizerV1 import BatchOptimizer
from Simulation_Classes_Single_ModeV4 import *
import matplotlib.pyplot as plt
#import tfq_quantum as tfq
import tensorflow as tf

In [ ]:
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#Modes Truncation
N= 15

In [ ]:
def one_mode_state(fock1, qubit_g = True): 
    '''
    Returns g x fock1 
    
    #takes N1 from global
    '''
    psi_1 = basis(N1,fock1) #initial state
    return tensor(basis(2,0), psi_1)

In [ ]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 50,#Batch size (number of circuit optimizations to run in parallel)
'epochs' : 100,#number of epochs before termination
'epoch_size' : 100,#number of adam steps per epoch
'learning_rate' : 0.001, #adam learning rate
'term_fid' : 0.999, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'initial_states' : [], #qubit tensor oscillator, start in |g> |0>
'target_states' : [], #end in |e> |target>.
'name' : 'Fock_stuff', #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}



#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

#### Filename for angles to be stored

In [ ]:
df = None 
fname = 'fock_prep'
columns = ['task', 'layer', 'pulse_time', 'BO_fid', 'filenum']

#2 files here are redundant
angles_filename_prefix = 'Data/angles'
opt_filename_prefix = 'Data/opt_details_'

In [ ]:
df = None

In [ ]:
from IPython.display import clear_output

In [ ]:
def main( n, start, df):
    '''
    Vary depth
    0->n state transfer
    start time
    df is data frame to store
    '''
    first_iter = True #this is for dataframe
    filenum = 0
    
    for n_ in range(1, n+1): #for |0n> -> |n0> transfer
        
        initial = one_mode_state(0)
        target = one_mode_state(n_)
        opt_params['initial_states'] = [initial]
        opt_params['target_states'] = [target]
        
        fid = 0
        layer =2
        
        while fid<(1-(1e-3)) : 
            clear_output(wait = True)            
            print('prev layer' + str(layer))
            print('prev fid' + str(fid))
            print(time.time()-start)
            print('--------')
            
            #optimizer
            opt_params['N_blocks'] = layer
            opt_params['name'] = opt_filename_prefix + str(filenum)
            opt = BatchOptimizer(**opt_params)
            #print(opt.filename)
            opt.optimize()
            #return opt
            BO_fid = opt.best_circuit()['fidelity']
            BO_fid = BO_fid.real
            angles_filename = angles_filename_prefix + '_' + str(filenum) + '.txt'
            opt.save_angles(filename = angles_filename)
            
            print('finished optimization')
            print(time.time()-start)
            print('-------')

            #pulses
            pulse_sim = ecd_pulse_single_mode(param_file = angles_filename) 
            pulse_sim.get_pulses()
            pulse_time =len(pulse_sim.cavity_dac_pulse_GHz)
            
            #qutip simulation
#             qutip_sim  = qutip_sim_two_mode(n_q = 2, n_c1 = N1,
#                                             n_c2 = N2, alpha1 = pulse_sim.alpha1,
#                                             alpha2 = pulse_sim.alpha2,
#                                             qubit_pulse = pulse_sim.qubit_dac_pulse_GHz)
#             qutip_sim.me_solve(initial = initial)
#             qutip_fid = qutip_sim.get_fidelity(target)
            
#             print('finished qutip_sim')
#             print(time.time()-start)
#             print('-------')
            
            df_new = pd.DataFrame([[n_, layer, pulse_time, BO_fid, filenum]], columns=columns)
            if first_iter : 
                first_iter = False
                df = df_new
            else:
                df = df.append(df_new, ignore_index = True)
            df.to_csv(fname, index=False)
            
            fid = BO_fid
            layer +=1
            filenum+=1
    return df

In [ ]:
start = time.time()
df = main(5, start, df)

In [ ]:
df.best_circuit()['fidelity']

# DUMP

In [ ]:
angles_filename_prefix

In [ ]:
angles_filename = angles_filename_prefix + '_1.txt'

In [ ]:
pulse_sim = ecd_pulse_two_mode(param_file = angles_filename)

In [ ]:
pulse_sim.betas

In [ ]:
params = np.loadtxt(angles_filename)
params

In [ ]:
np.ndim(params)

In [ ]:
np.asarray([complex(params[0][i], params[1][i]) for i in range(len(params[0]))])

In [ ]:
#can print info, including the best circuit found.
opt.print_info() 

In [ ]:
angles_filename = 'test_opt_angles.txt'
opt.save_angles(filename = angles_filename)

# Get Pulses

In [ ]:
from Simulation_Classes_Two_ModeV2 import *
# Classes: ecd_pulse_single_mode to get pulses
#          , qutip_sim_single_mode to simulate on qutip

In [ ]:
T1_mode1 = 2e+6 #2ms
T1_mode2 = 2e+6 #2ms

pulse_sim = ecd_pulse_two_mode(param_file = angles_filename,
                              kappa1 = 1/T1_mode1,
                              kappa2 = 1/T1_mode2)

In [ ]:
pulse_sim.get_pulses()

In [ ]:
pulse_sim.plot_pulses()

# Qutip Simulation

In [ ]:
qutip_sim  = qutip_sim_two_mode(n_q = 2, n_c1 = 10, n_c2 = 10, alpha1 = pulse_sim.alpha1, alpha2 = pulse_sim.alpha2, qubit_pulse = pulse_sim.qubit_dac_pulse_GHz)

In [ ]:
# Add various terms 

# to the hamiltonian 
qutip_sim.add_bare_qubit_mode_coupling() # chi/2 adag a sigmaz
qutip_sim.add_mode_mode_coupling()
qutip_sim.add_stark_shift() # chi/2 alpha^2 sigmaz

# to the collapse operators (noise)
qutip_sim.add_qubit_relaxation(T1 = 30e+3) # T1 in nanoseconds
qutip_sim.add_qubit_dephasing(T1 = 30e+3, Techo = 50e+3) # T1, T2 in nanoseconds
qutip_sim.add_cavity_relaxation(T1_mode1 = 2e+6, T1_mode2 = 2e+6) # assumes kappa correction to equation of motion in pulses
qutip_sim.add_cavity_dephasing(T1_mode1 = 2e+6, Techo_mode1 = 3e+6, T1_mode2 = 2e+6, Techo_mode2 = 3e+6) # T1, T2 from photon blockade paper


In [ ]:
qutip_sim.me_solve(initial = initial)  #initial state by default is g0

In [ ]:
qutip_sim.plot_populations()

In [ ]:
qutip_sim.get_fidelity(target)